# JEM092 Asset Pricing - Homework 2

### Group 82780095

Petr Dvořáček, Jan Kubal, Matyáš Mattanelli

---

## Abstract

## 1. Introduction

## 2. Data Description

In this section we provide an overview of the utilized data. We examine summary staistics of the variables as well as pairwise correlations. Lastly, we perform a persistence analysis. We start by loading the data along with the necessary packages.

In [1]:
#Loading the packages
#install.packages("xts", "readr", "lubridate", "quantmod", "moments", "sandwich", "lmtest")
suppressPackageStartupMessages({
    library(xts) #Handling xts objects
    library(readr) #Reading csv files
    library(lubridate) #Date manipulation
    library(quantmod) #Returns calculation
    library(moments) #Skewness and kurtosis functions
    library(sandwich) #Robust covariance matrix estimation
    library(lmtest) #Robust covariance matrix estimation
})

#Loading the data
monthly_returns <- readRDS("Final data/monthly_excess_returns.RData")
skewness_data <- readRDS("Final data/skewness.RData")
betas <- readRDS("Final data/betas.RData")
market_cap <- readRDS("Final data/market_cap_monthly.RData")
size <- readRDS("Final data/size.RData")
fffactors <- readRDS("Final data/fffactors_monthly.RData")
momentum <- readRDS("Final data/momentum.RData")
liquidity <- readRDS("Final data/liquidity.RData")

Before proceeding to the summary statistics, we need to take a look at the inspected period. The required time span according to the first homework is 01.01.2007 - 28.02.2022. However, in the second assignment the required period is 01.01.2007 - 31.12.2020. We select the former in order to utilize as much data as possible. In addition, the data for the Market Capitalization variable are available from 28.02.2009 onwards which decreases the usable time span. Furthermore, the liquidity measure ends in December 2021 which shrinks our data even further. Nevertheless, for the purpose of data description, we retain all values for all variables available after 01.01.2007 and until 28.02.2022. We will make the proper adjustments later in the empirical analysis.

In [2]:
#Restricting the period (01/01/2007 - 28/02/2022)
market_cap <- market_cap["/2022-02-28"]
size <- size["/2022-02-28"]
fffactors <- fffactors["/2022-02-28"]
momentum <- momentum["/2022-02-28"]

Finally, we can proceed with describing the data. Our final data set contains monthly observations for 250 stocks in the period between 01.01.2007 and 28.02.2022. The three sorting variables that we inspect are Beta, Size, and Skewness. In addition, the dependent variable in our setting are the one-ahead excess returns. We also utilize Market Capitalization for value-weighted portfolios, and therefore we provide the statistics for this variable as well. For each variable and each period we calcute 12 statistics. We then take an average of these values over all available periods. In order to facilitate the process, we define a custom function.

In [3]:
#Defining a function to calculate the summary statistics
calc_sum_stats <- function(var_data) {
  period_stats <- matrix(ncol = 12, nrow = nrow(var_data)) #A matrix for the statistics from each period
  perc_5 <- function(x, na.rm) {return(quantile(x, probs = 0.05, na.rm = na.rm))} #Function to calculate the 5th percentile of data
  perc_25 <- function(x, na.rm) {return(quantile(x, probs = 0.25, na.rm = na.rm))} #Function to calculate the 25th percentile of data
  perc_75 <- function(x, na.rm) {return(quantile(x, probs = 0.75, na.rm = na.rm))} #Function to calculate the 75th percentile of data
  perc_95 <- function(x, na.rm) {return(quantile(x, probs = 0.95, na.rm = na.rm))} #Function to calculate the 25th percentile of data
  calc_n <- function(x, na.rm) {return(sum(!is.na(x)))} #Function to calculate the number of observations
  funcs <- list(mean, sd, skewness, kurtosis, min, perc_5, perc_25, median, perc_75, perc_95, max, calc_n) #Store the functions we will use
  for (i in 1:nrow(var_data)) { #Loop through the periods
    for (j in 1:length(funcs)) { #Loop through the functions
      period_stats[i, j] <- funcs[[j]](as.numeric(var_data[i, ]), na.rm = T) #Apply each function and store the result 
    }
  }
  time_avg <- apply(period_stats, 2, mean, na.rm = T) #Take a time series mean of each statistic
  return(round(time_avg, 2)) #Round to two decimals and return the result
}

#Applying the function
sum_stats <- as.data.frame(matrix(nrow = 5, ncol = 12)) #Place holder for the final summary statistics
colnames(sum_stats) <- c("Mean", "Sd", "Skewness", "Kurtosis", "Min", "5%", "25%", "Median", "75%", "95%", "Max", "n") #Rename the columns as the statistics
var_names <- c("Returns", "Skewness", "Beta", "Size", "Market cap") #Define a vector of variable names
row.names(sum_stats) <- var_names #Rename the row names as the variables
data_list <- list(monthly_returns, skewness_data, betas, size, market_cap) #Store the data in a list for looping
for (i in 1:5) { #Loop through the variables and apply the function
  sum_stats[i, ] <- calc_sum_stats(data_list[[i]])
}

The resulting summary statistics are available below. Each row shows the 12 statistics for each of the 5 variables. A brief glance at the first row reveals that the average excess return in our data set is positive. Both the mean and median are identical suggesting a symetric distribution. This is confirmed by the skewness (first row, third column) which is quite close to zero. The value is slightly positive which likely reflects that fact that there is a lower bound on returns (-100%) but no upper bound. The value for kurtosis on the other hand is well above 3 (kurtosis of normal distribution) and therefore, the distribution of the returns is likely heavy-tailed. The maximum and minimum excess returns are 35% and -23%, respectively. The maximum value, however, is likely an outlier given the relatively low value of the 95th percentile. 

As for the Skewness sorting variable, it seems to be distributed symmetrically around zero. This confirms our previous observations that the returns are not substantially skewed. It does attain some extreme values at both tails.

A second sorting variable of interest is the stocks' sensitivity to market. As we can see, the average value is around one which indicates a perfect reaction to any market development. In addition, it does not attain a single negative value suggesting that all the stocks react positively to the market. The distribution of Betas seem to be quite close to normal.

The Market Capitalization seems to be very far from normal distribution, however, the logarithmic transformation seems to remedy this issue since the Size variable is quite close to normal distribution in terms of both skewness and kurtosis. The positive skewness of Market Capitalization is caused by some large outliers in the upper tail which signals that there are one or a few firms that cover a substantially larger portion of the market than the rest. The maximum value is very far from the 95% percentile. Once again, Size does not seem to suffer from these issues since the logarithmic tranasformation decreases the emphasis of outliers.

Overall, we can see that in the average period we have majority of the observations for all stocks (denoted by the last column) with the exception of the Market Capitalization and Size variables which seem to have slightly more missing observations. 

In [4]:
#Summary statistics
sum_stats

,Mean,Sd,Skewness,Kurtosis,Min,5%,25%,Median,75%,95%,Max,n
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Returns,0.01,0.07,0.50,7.39,-0.23,-0.09,-0.03,0.01,0.05,0.13,0.35,248.78
Skewness,0.02,0.97,0.25,18.29,-4.92,-1.22,-0.35,-0.03,0.34,1.45,5.41,248.64
Beta,1.02,0.37,0.39,3.34,0.21,0.45,0.76,1.00,1.25,1.65,2.24,248.64
Size,2.99,1.12,0.60,3.24,0.43,1.47,2.18,2.84,3.63,5.08,6.56,244.67
Market cap,45.48,90.88,5.21,38.28,1.82,5.06,9.77,18.46,40.71,168.12,858.28,245.14


Moving on from the summary statistic, we provide the average pairwise correlations between the variables in question. Similarly as in the previous case, we firstly calculate the cross-sectional correlations between the variables and then average the values across periods.

In [5]:
#Defining a function to calculate the average correlation between two variables
calc_corr <- function(var1, var2, method = "pearson") { #Expects data for both variables and an indicator for type (default is pearson)
  var1 <- var1[index(var1) %in% index(var2)] #Filter only data available for both variables
  cross_sec_corrs <- rep(NA, nrow(var1)) #Empty vector the results of each period
  for (i in 1:nrow(var1)) { #Loop through the periods available for the first variable
    cross_sec_corrs[i] <- cor(as.numeric(var1[i]), as.numeric(var2[index(var1)[i]]), use = "pairwise.complete.obs", method = method) #Calculate the correlation for each period
  }
  return(round(mean(cross_sec_corrs, na.rm = T), 2)) #Return the rounded time series average of the correlations
}

#Applying the function
corrs <- as.data.frame(matrix(nrow = 5, ncol = 5)) #Empty matrix for the results
colnames(corrs) <- var_names #Column names
row.names(corrs) <- var_names #Row names
indices <- combn(1:5, 2) #Get the indices for each pair (unique and excluding pairing variables with themselves)
for (i in 1:ncol(indices)) { #Loop through the pairs
  corrs[indices[1, i], indices[2, i]] <- calc_corr(data_list[[indices[1, i]]], data_list[[indices[2, i]]], method = "spearman") #Spearman correlation in the upper triangle
  corrs[indices[2, i], indices[1, i]] <- calc_corr(data_list[[indices[2, i]]], data_list[[indices[1, i]]]) #Pearson correlation in the lower triangle
}

The results are available in the table below. The diagonal contains missing values since the correlation of a variable with itself is 1 and as such is not relevant. Above the diagonal we present the Spearman correlation while below is the Pearson correlation. As can be seen, the values are largely similar with the exception of Size and Market Capitalization since the Spearman correlation captures the perfectly monotonic relationship while Pearson correlation shows only the degree of linear dependence. As for the remaining values, we can see that the variable are mostly uncorrelated. Nevertheless, the Skewness measure seems to be slightly positively correlated with the Returns and with Beta. In addition, Size and Beta appear to be negatively correlated suggesting that larger stocks are less market-sensitive.

In [6]:
#Pairwise correlations
corrs

,Returns,Skewness,Beta,Size,Market cap
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Returns,NA,0.10,0.02,-0.01,-0.01
Skewness,0.13,NA,0.14,0.01,0.01
Beta,0.03,0.11,NA,-0.13,-0.13
Size,-0.01,0.02,-0.14,NA,1.00
Market cap,0.00,0.03,-0.08,0.75,NA


Lastly, we perform the persistence analysis. We consider five lags and present the average autocorrelation over all periods for a given lag for each variable.

In [7]:
#Defining a function to calculate persistence of a variable
calc_pers <- function(var_data, lags = 5) { #Expects an xts object and the number of lags how far to calculate the correlations
  final_persist <- rep(NA, lags) #A vector to store the results 
  for (i in 1:lags) { #Loop through the lags
    cross_persist <- rep(NA, nrow(var_data)) #A vector to store the results from each period
    for (j in 1 :nrow(var_data)) { #Loop through the periods
      current_period <- index(var_data)[j] #Store the current period
      lagged_period <- as.Date(as.yearmon(current_period %m-% months(i)), frac = 1) #Derive the lagged period
      if (!lagged_period %in% index(var_data)) { #If the lag is unavailable, skip
        next
      }
      cross_persist[j] <- cor(as.numeric(var_data[j]), as.numeric(var_data[lagged_period]), use = "pairwise.complete.obs") #Calculate Pearson between the variable and the appropriate lag
    }
    final_persist[i] <- mean(cross_persist, na.rm = T) #Calculate the time series mean
  }
  return(round(final_persist, 3)) #Return the rounded correlations
}

#Apply the function
lags <- 5 #Specify the required number of lags
persistence <- as.data.frame(matrix(ncol = lags, nrow = 5)) #Data frame for the results (lags in columns, vars in rows)
colnames(persistence) <- paste("Lag ", 1:lags) #Rename the columns with lag numbers
row.names(persistence) <- var_names #Rename the rows with variable names
for (i in 1:nrow(persistence)) { #Loop through the variables and calculate their persistence
  persistence[i, ] <- calc_pers(data_list[[i]], lags = lags) #Apply the function
}

The results of the table below seem to agree with basic intuition. The Returns do not show any significant autocorrelations which demostrates their unpredictable behavior. On the other hand, the remaining variables seem to be quite persitent. In case of Skewness, the persitence is slowly decaying which is likely mechanical since lags that are close to each other contain larger portions of the same data for calculation. We cannot observe any strong decay for Beta, Size, and Market Cap which suggests that the market sensitivity of a stock does not change much over time on average and the same applies to its market share. 

In [8]:
#Persistence analysis
persistence

,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Returns,-0.033,0.020,-0.016,0.004,-0.010
Skewness,0.906,0.802,0.721,0.643,0.561
Beta,0.988,0.972,0.951,0.939,0.920
Size,0.998,0.996,0.994,0.992,0.990
Market cap,0.998,0.996,0.994,0.992,0.989


## 3. Methodology

This section presents an overview of the utilized methodology. We also introduce the functions used to perform the analyses. These include the univariate portfolio analysis, bivariate portfolio analysis, and Fama-MacBeth regression.

The univariate portfolio analysis inspects the relationship between a single indenpendent variable and a dependent variable. The dependent variable in our case are the one-ahead returns. We separately investigate three independent variables: Beta, Size, and Skewness. The nature of the analysis allows us to not make any assumptions regarding the linearity of the relationship.

For each variable and each period we divide the stocks into 5 portfolios. These are formed based on breakpoints calculated as the 20th, 40%, 60%, and 80% percentile of the sorting variable. When allocating the stocks into the portfolios we consider closed intervals to prevent the existence of empty portfolios. This may result in some stock being allocated to multiple portfolios. This approach follows the textbook by Bali *et al.* (2016). For each portfolio we then calculate the cross-sectional average of the sorting variable as well as the one-ahead returns. We consider both an equally weighted average as well as value-weighted average. Subsequently, we calculate the difference portfolio as the difference between the average values of the last and first portfolios.

After obtaining the cross-sectional averages for each period, we calculate the time-series average for each portfolio. We report the *t*-statistic associated with the values adjusted using the robust Newey-West covariance matrix with 6 lags to ensure robustness to serial correlation. In addition, we use five factors to obtain Fama-French-Carhart-Pastor-Stambaugh alpha and its *t*-statistic. The five factors include the Market premium, High Minus Low, Small Minus Big, Momentum, and Liquidity. The value of alpha represents the risk-adjusted returns.

In [9]:
#Defining a function to perform a univariate sort
univariate_sort <- function(sort_variable, no_of_ports = 5, weighted = F) {
  sort_variable_name <- gsub("MDLZ.", "", colnames(sort_variable)[1]) #Extracting the name of the sorting variable
  final_ouptut <- as.data.frame(matrix(ncol = no_of_ports + 1, nrow = 5)) #Data frame for the final output
  colnames(final_ouptut) <- c(1:no_of_ports, paste(no_of_ports, 1, sep = "-")) #Column names for clarity
  row.names(final_ouptut) <- c(paste0("Mean of ", sort_variable_name), "Mean of one-ahead returns", "T-statistic", "Alpha", "T-statistic (alpha)")
  all_avg_sort_values <- c() #Empty vector to which we will append the results from each period (sort variable)
  all_avg_returns <- c() #Empty vector to which we will append the results from each period (returns)
  for (i in 1:nrow(sort_variable)) { #Loop through the rows (periods)
    current_period <- index(sort_variable)[i] #Store the current period
    next_period <- as.Date(as.yearmon(current_period %m+% months(1)), frac = 1) #Store the next period (for the 1-ahead returns)
    if (!next_period %in% index(monthly_returns)) { #In case we do not have 1-ahead returns, skip the period
      next
    }
    average_sort_values <- rep(NA, no_of_ports) #Place holder for the cross-sectional average values of the sort variable
    average_returns <- xts(matrix(nrow = 1, ncol = no_of_ports + 1), order.by = current_period) #Place holder for the cross-sectional average values of the 1-ahead returns
    breakpoints <- quantile(sort_variable[current_period], probs = seq(1/no_of_ports, 1 - 1/no_of_ports, 1/no_of_ports), na.rm = T) #Find the breakpoints
    for (j in 1:no_of_ports) { #Looping through the portfolios
      if (j == 1) { #For the first portfolio we have only one condition
        indic <- which(sort_variable[current_period] <= breakpoints[j]) #Find the stocks in the portfolio and save their column indices
      } else if (j == no_of_ports) { #For the last portfolio we have only one condition as well
        indic <- which(sort_variable[current_period] >= breakpoints[j -  1])
      } else { #The rest of the portfolios (two conditions)
        indic <- which(sort_variable[current_period] >= breakpoints[j - 1] & sort_variable[current_period] <= breakpoints[j]) #"=" at both inequalities to prevent empty portfolios
      }
      if (weighted == T) { #Market cap weighted average
        mc_weights <- as.numeric(market_cap[current_period, indic]) #Storing the weights
        if (length(mc_weights) == 0) { #If we do not have the market cap data, we have to skip
          next
        }
        mc_weights[is.na(mc_weights)] <- 0 #0 value for missings = they have no weight
        average_returns[, j] <- weighted.mean(as.numeric(monthly_returns[next_period, indic]), w = mc_weights, na.rm = T)
      } else { #Equally-weighted average
        average_returns[, j] <- mean(monthly_returns[next_period, indic], na.rm = T) #Calculate the average of 1-ahead returns
      }
      average_sort_values[j] <- mean(sort_variable[current_period, indic], na.rm = T) #Calculate the average of the sort variable for the given portfolio
    }
    average_returns[, no_of_ports + 1] <- average_returns[, no_of_ports] - average_returns[, 1] #The difference portfolio
    all_avg_sort_values <- rbind(all_avg_sort_values, average_sort_values) #Append the results
    all_avg_returns <- rbind(all_avg_returns, average_returns) #Append the results
  }
  final_ouptut[1, ] <- c(apply(all_avg_sort_values, 2, mean, na.rm = T) , NA) #Final average values of the sort variable
  for (i in 1:(no_of_ports + 1)) { #Loop through the portfolios and get the time series avg of returns, t-stat and alpha
    model_avg <- lm(na.omit(all_avg_returns[, i]) ~ 1) #Regress returns of each portfolio on the intercept (for weighted portfolio there may be missing values and coeftest cannot handle them => remove)
    model_avg_adj <- coeftest(model_avg, vcov = NeweyWest(model_avg, lag = 6)) #Make Newey-West adjustment of std errors
    final_ouptut[2, i] <- model_avg_adj[1, 1] #Store the intercept (=time series average)
    final_ouptut[3, i] <- model_avg_adj[1, 3] #Store the t-statistic
    #Regress the returns on the five factors
    factor_data <- na.omit(merge.xts(all_avg_returns[, i], fffactors[, 1:3], momentum, liquidity)) #Merge the data needed for the factor model (to secure the same number of obs) + remove NAs since coeftest cannot handle it
    factor_model <- lm(factor_data[, 1] ~ factor_data[, 2] + factor_data[, 3] + factor_data[, 4] + factor_data[, 5] + 
                         factor_data[, 6])
    factor_model_adj <- coeftest(factor_model, vcov = NeweyWest(factor_model, lag = 6)) #Make Newey-West adjustment
    final_ouptut[4, i] <- factor_model_adj[1, 1] #Store the intercept (alpha)
    final_ouptut[5, i] <- factor_model_adj[1, 3] #Store the t-statistic
  }
  return(final_ouptut)
}

Apart of the univariate sort, we also perform an independent bivariate portfolio analysis. In this way we can examine the relation between one independent variable and the dependent variable after controling for some other variable. In our case we consider a sort on Size and Beta and then on Size and Skewness. The procedure is quite similar to the univariate sort.

In our analysis we divide both sorting variables into three groups based on the 30th and 70th percentile. We then form a portfolio for all possible combinations resulting in 16 portfolios in total (we also include the difference portfolios). As in the previous case, we consider close intervals to prevent some portfolios to be empty. Since we are concerned with an independent sorting procedure, we find the breakpoints for each variable independently and then find the stocks belonging to each portfolio by combining the breakpoints. Then as in the univariate sort, we calculate the cross-sectional averages for each portfolio in each period. Subsequently, we calculate the difference portfolios and lastly the difference in difference portfolio.

Once we have the time series of excess returns for each portfolio, the procedure is identical to the univariate sort. The only exception is that we do not report the average values of the sorting variables.

In [10]:
#Defining a function to perform a bivariate sort (independent and weighted)
bivariate_sort <- function(sort_variable1, sort_variable2, percs1 = c(0.3, 0.7), percs2 = c(0.3, 0.7)) { #Accepts two sort variables data and required percentiles for the portfolios
  no_of_ports1 <- length(percs1) + 1 #Storing the number of portfolios of the first sort variable
  no_of_ports2 <- length(percs2) + 1 #Storing the number of portfolios of the second sort variable
  sort_variable1_name <- gsub("MDLZ.", "", colnames(sort_variable1)[1]) #Extracting the name of the first sorting variable
  sort_variable2_name <- gsub("MDLZ.", "", colnames(sort_variable2)[1]) #Extracting the name of the second sorting variable
  final_ouptut <- as.data.frame(matrix(ncol = no_of_ports1 + 1, nrow = 4*(no_of_ports2 + 1))) #Data frame for the final output
  colnames(final_ouptut) <- c(paste0(sort_variable1_name, " ", 1:no_of_ports1), paste0(sort_variable1_name, " ", paste0(no_of_ports1, "-", 1))) #Column names for clarity
  sort_variable2_ports <- c(paste0(sort_variable2_name, " ", 1:no_of_ports2), paste0(sort_variable2_name, " ", paste0(no_of_ports2, "-", 1))) #Portfolios of the second sort variable stored to use for row names of the final output
  statistics <- c("Mean", "T-statistic", "Alpha", "T-statistic (alpha)") #Statistics to paste for row names
  row.names(final_ouptut) <- as.vector(t(outer(sort_variable2_ports, statistics, FUN = "paste", sep = " - "))) #Row names for clarity
  all_avg_returns <- list() #Empty vector to which we will append the matrices of results from each period (returns)
  find_indics <- function(xts_object, percs) { #A function that returns boolean vectors for given quantiles of given data
    breakpoints <- quantile(xts_object, probs = percs, na.rm = T) #Find the breakpoints
    no_of_quants <- length(breakpoints) + 1 #Store the number of quantiles
    indics <- vector("list", no_of_quants) #Empty list to store the final indices
    for (i in 1:no_of_quants) { #Loop through the quantiles
      if (i == 1) { #For the first quantile we have only one condition
        indic <- xts_object <= breakpoints[i] #Find the stocks in the first quantile
      } else if (i == no_of_quants) { #For the last quantile we have only one condition as well
        indic <- xts_object >= breakpoints[i -  1]
      } else { #The rest of the quantiles (two conditions)
        indic <- xts_object >= breakpoints[i - 1] & xts_object <= breakpoints[i] #"=" at both inequalities to prevent empty quantiles (see book)
      }
      indics[[i]] <- indic
    }
    return(indics)
  }
  for (i in 1:nrow(sort_variable1)) { #Loop through the rows (periods) of the first sorting variable (we are interested in the intersection of the periods so we can loop through just one of the indices)
    current_period <- index(sort_variable1)[i] #Store the current period
    next_period <- as.Date(as.yearmon(current_period %m+% months(1)), frac = 1) #Store the next period (for the 1-ahead returns)
    if (!next_period %in% index(monthly_returns) | !current_period %in% index(sort_variable2)) { #In case we do not have 1-ahead returns or the values for the second variable, skip the period
      next
    }
    average_returns <- matrix(nrow = no_of_ports2 + 1, ncol = no_of_ports1 + 1) #Place holder for the cross-sectional average values of the 1-ahead returns
    indics1 <- find_indics(sort_variable1[current_period], percs1) #Divide the first sorting variable into quantiles (get the boolean values for the stocks)
    indics2 <- find_indics(sort_variable2[current_period], percs2) #Divide the second sorting variable into quantiles (get the boolean values for the stocks)
    indics_comb <- expand.grid(indics1, indics2) #Make all possible combinations for all quantiles of both variables
    indics_for_mat <- expand.grid(1:no_of_ports1, 1:no_of_ports2) #Indices for writing the returns in the matrix for each period
    for (j in 1:nrow(indics_comb)) { #Loop through the portfolios
      final_indics <- which(unlist(indics_comb[j, 1]) & unlist(indics_comb[j, 2])) #Find the column indices for a given portfolio
      mc_weights <- as.numeric(market_cap[current_period, final_indics]) #Storing the market cap weights
      if (length(mc_weights) == 0) { #If we do not have the market cap data, we have to skip
        next
      }
      mc_weights[is.na(mc_weights)] <- 0 #0 value for missings = they have no weight
      average_returns[indics_for_mat[j, 2], indics_for_mat[j, 1]] <- weighted.mean(as.numeric(monthly_returns[next_period, final_indics]), w = mc_weights, na.rm = T)  
    }
    for (j in 1:(ncol(average_returns)- 1)) { #Calculate the difference portfolios of the second sorting variable (last row)
      average_returns[nrow(average_returns), j] <- average_returns[nrow(average_returns) - 1, j] - average_returns[1, j] 
    }
    for (j in 1:nrow(average_returns)) { #Calculate the difference portfolios of the first sorting variable + the diff in diff portfolio (last column)
      average_returns[j, ncol(average_returns)] <- average_returns[j, ncol(average_returns) - 1] - average_returns[j, 1]
    }
    all_avg_returns[[length(all_avg_returns) + 1]] <- average_returns #Append the matrix with results for the current period
    names(all_avg_returns)[length(all_avg_returns)] <- as.character(current_period) #Name the element in the list with the period for later usage
  }
  indics_for_ports <- expand.grid(1:(no_of_ports1 + 1), 1:(no_of_ports2 + 1)) #Store the indices for each portfolio in the returns matrix for the loop
  final_returns <- rep(NA, ((no_of_ports1 + 1)*(no_of_ports2 + 1))) #A vector to store the final returns for each portfolio
  final_t_stats <- rep(NA, ((no_of_ports1 + 1)*(no_of_ports2 + 1))) #A vector to store the final t-statistics for each portfolio
  final_alphas <- rep(NA, ((no_of_ports1 + 1)*(no_of_ports2 + 1))) #A vector to store the final alphas for each portfolio
  final_t_stats_alphas <- rep(NA, ((no_of_ports1 + 1)*(no_of_ports2 + 1))) #A vector to store the final t-statistics for alphas for each portfolio
  for (i in 1:((no_of_ports1 + 1)*(no_of_ports2 + 1))) { #Loop through the portfolios and get the time series avg of returns, t-stat and alpha
    data_to_use <- xts(unlist(lapply(all_avg_returns, "[", indics_for_ports[i, 2], indics_for_ports[i, 1])), order.by = as.Date(names(all_avg_returns))) #Get the time series for each portfolio
    model_avg <- lm(na.omit(data_to_use) ~ 1) #Regress returns of each portfolio on the intercept (for weighted portfolio there may be missing values and coeftest cannot handle them => remove)
    model_avg_adj <- coeftest(model_avg, vcov = NeweyWest(model_avg, lag = 6)) #Make Newey-West adjustment of std errors
    final_returns[i] <- model_avg_adj[1, 1] #Store the intercept (=time series average)
    final_t_stats[i] <- model_avg_adj[1, 3] #Store the t-statistic
    #Regress the returns on the five factors
    factor_data <- na.omit(merge.xts(data_to_use, fffactors[, 1:3], momentum, liquidity)) #Merge the data needed for the factor model (to secure the same number of obs) + remove NAs since coeftest cannot handle it
    factor_model <- lm(factor_data[, 1] ~ factor_data[, 2] + factor_data[, 3] + factor_data[, 4] + factor_data[, 5] + 
                         factor_data[, 6])
    factor_model_adj <- coeftest(factor_model, vcov = NeweyWest(factor_model, lag = 6)) #Make Newey-West adjustment
    final_alphas[i] <- factor_model_adj[1, 1] #Store the intercept (alpha)
    final_t_stats_alphas[i] <- factor_model_adj[1, 3] #Store the t-statistic
  }
  final_result <- list(final_returns, final_t_stats, final_alphas, final_t_stats_alphas)
  for (i in 1:4) { #Loop through the list of results
    results_mat <- matrix(final_result[[i]], ncol = no_of_ports1 + 1, byrow = T) #Reshape the vector to matrix to facilitate filling in the final output
    final_ouptut[seq(i, nrow(final_ouptut), 4), ] <- results_mat #Fill the appropriate rows with results for each statistic
  }
  return(final_ouptut)
}

The last part of our analysis entails performing the Fama-Macbeth regression. In this method we make the assumption that the relationship between the variables in question is linear. We firstly consider the sole relationship between Skewness and excess returns and then we also include the variable Size and Beta into the regression.

The procedure is somehow similar to the portfolio analysis in the way that we first estimate individual cross-sectional regressions for each period, we store the coefficients, and then we calculate the average over all periods. In this case we present only the robust *t*-statistics and also the average Adjusted R-squared.

In [23]:
#Defining a function to perform the Fama-Macbeth regression
fama_macbeth <- function(indep_var) { #Expects a list of xts objects with independent variables
  cross_sec_coefs <- c() #Empty vector to which we will append the coefficients for each period
  adj_r_squared <- c() #Empty vector for adjusted R-squared in each period
  no_of_obs <- c() #Empty vector to store the number of observations in each period
  for (i in 1:nrow(monthly_returns)) { #Loop through the periods
    next_period <- index(monthly_returns)[i] #Store the period from which the one ahead returns come
    current_period <- as.Date(as.yearmon(next_period %m-% months(1)), frac = 1) #Store the period for the independent variables
    move_to_next_period <- F #Setting up an indicator for incomplete periods
    for (j in 1:length(indep_var)) { #Check if the current period is contained in all independent variables
      if (current_period %in% index(indep_var[[j]])) { #If it is, move to next variable
        next
      } else { #If it is not, set the indicator and break
        move_to_next_period <- T
        break
      }
    }
    if (move_to_next_period == T) { #If one of the independent variables does not contain the required period, move to next period
      next
    }
    cross_sec_data <- as.data.frame(matrix(ncol = length(indep_var) + 1, nrow = ncol(monthly_returns))) #Data holder for each period
    colnames(cross_sec_data) <- c("Returns", names(indep_var)) #Column names that will be used in the formula
    cross_sec_data[, 1] <- as.numeric(monthly_returns[next_period]) #Fill in the data for the dependent variable
    for (j in 1:length(indep_var)) { #Fill in the data for each independent variable
      cross_sec_data[, j + 1] <- as.numeric(indep_var[[j]][current_period])
    }
    form <- paste("Returns", paste(colnames(cross_sec_data)[2:ncol(cross_sec_data)], collapse = "+"), sep = "~") #Define the formula
    cross_sec_model <- lm(form, data = cross_sec_data) #Estimate the model
    cross_sec_coefs <- rbind(cross_sec_coefs, cross_sec_model$coefficients) #Append the coefficients
    adj_r_squared[length(adj_r_squared) + 1] <- summary(cross_sec_model)$adj.r.squared #Append the adjusted r_squared
    no_of_obs[length(no_of_obs) + 1] <- nrow(na.omit(cross_sec_data)) #Append the number of observations
  }
  final_output <- as.data.frame(matrix(nrow = 2, ncol = ncol(cross_sec_coefs) + 2)) #Place holder to store the final output (time series means and t-statistics + adjusted r squared and no of obs)
  colnames(final_output) <- c(colnames(cross_sec_coefs), "Adj. R-squared", "n") #Rename the columns for clarity
  row.names(final_output) <- c("Mean", "T-statistic") #Rename the rows for clarity
  for (i in 1:ncol(cross_sec_coefs)) { #Calculate the time series mean and t-statistics for each coefficient
    time_model <- lm(cross_sec_coefs[, i] ~ 1) #Regress each time series on a constant to get the mean estimate
    time_model_adj <- coeftest(time_model, vcov = NeweyWest(time_model, lag = 4)) #Newey-West adjustment of the errors
    final_output[1, i] <- time_model_adj[1, 1] #Store the mean
    final_output[2, i] <- time_model_adj[1, 3] #Store the standard error
  }
  final_output[1, ncol(cross_sec_coefs) + 1] <- mean(adj_r_squared, na.rm = T) #Calculate the average adjusted R-squared
  final_output[1, ncol(cross_sec_coefs) + 2] <- mean(no_of_obs, na.rm = T) #Calculate average number of observations
  return(final_output)
}

## 4. Results

In this section we present the results of our analyses. We start with the univariate sort.

### Univariate sort

The first sorting variable of interest is Beta. It measures the stock's sensitivity to market. The table below which presents the results of the univariate sort. The average values of the sorting variable are strictly increasing for each portfolio which indicates that the procedure should be correct.

In [12]:
#Univariate sort on betas (equally-weighted)
univariate_sort_betas <- univariate_sort(betas) #Equally weighted
univariate_sort_betas

,1,2,3,4,5,5-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mean of Beta,0.53202163,0.81497291,1.00348952,1.20147665,1.55994054,NA
Mean of one-ahead returns,0.01143895,0.01198626,0.01372895,0.01920353,0.02537157,0.01393262
T-statistic,2.86306509,2.68074422,2.26730692,2.74103627,2.77506299,1.58604641
Alpha,0.01551493,0.01370149,0.01590587,0.02076138,0.02759206,0.01207713
T-statistic (alpha),3.75423832,3.18190263,3.02882490,3.13358506,3.01850822,1.35356534


In [13]:
#Univariate sort on betas (value-weighted)
univariate_sort_weighted_betas <- univariate_sort(betas, weighted = T) #Market cap weighted (rbind warning should be okay, just NAs)
univariate_sort_weighted_betas

Warning message in rbind(deparse.level, ...):
"mismatched types: converting objects to numeric"


,1,2,3,4,5,5-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mean of Beta,0.52523558,0.81131749,1.00341777,1.20227214,1.56215054,NA
Mean of one-ahead returns,0.01478213,0.01628415,0.01775658,0.02161191,0.01960544,0.0048233133
T-statistic,3.99620608,4.07157476,3.06100328,3.81349682,2.71376673,0.6650608903
Alpha,0.02063223,0.01873594,0.01929609,0.02071345,0.02041098,-0.0002212442
T-statistic (alpha),5.54768786,3.86770990,3.53098803,3.34236564,2.33016626,-0.0281559398


In [14]:
#Univariate sort on size (equally weighted)
univariate_sort_size <- univariate_sort(size)
univariate_sort_size

,1,2,3,4,5,5-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mean of Size,1.60434604,2.29910489,2.82673288,3.43900684,4.71052537,NA
Mean of one-ahead returns,0.03038219,0.01766046,0.01638934,0.01430399,0.01414767,-0.01623452
T-statistic,5.28528435,3.66600463,3.66624437,3.55093346,3.72380327,-6.15070719
Alpha,0.03261987,0.01917303,0.01877600,0.01606425,0.01664241,-0.01597746
T-statistic (alpha),4.65950303,3.37794216,3.27686290,3.15232484,3.74793317,-5.10901045


In [15]:
#Univariate sort on size (value-weighted)
univariate_sort_weighted_size <- univariate_sort(size, weighted = T) #Market cap weighted
univariate_sort_weighted_size

,1,2,3,4,5,5-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mean of Size,1.60434604,2.29910489,2.82673288,3.43900684,4.71052537,NA
Mean of one-ahead returns,0.02825910,0.01729543,0.01603647,0.01436015,0.01487153,-0.01338757
T-statistic,5.25354607,3.59162958,3.66541383,3.56806962,3.79019379,-4.50971406
Alpha,0.03086117,0.01872467,0.01855382,0.01635925,0.01685969,-0.01400149
T-statistic (alpha),4.58008318,3.31482304,3.24020014,3.24800568,3.94259297,-3.70567811


In [16]:
#Univariate sort on skewness (equally weighted)
univariate_sort_skewness <- univariate_sort(skewness_data)
univariate_sort_skewness

,1,2,3,4,5,5-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mean of Skewness,-1.10907322,-0.28951433,-0.03510382,0.24827907,1.21044639,NA
Mean of one-ahead returns,0.01508861,0.01720480,0.01479196,0.01676539,0.01798657,0.002897961
T-statistic,3.10439400,3.12824533,2.55045516,2.59750882,2.98106799,0.964451126
Alpha,0.01857681,0.01947574,0.01669970,0.01903557,0.01977279,0.001195983
T-statistic (alpha),3.85872199,4.34398306,3.02933078,3.07137440,3.53794126,0.470713946


In [17]:
#Univariate sort on skewness (value-weighted)
univariate_sort_weighted_skewness <- univariate_sort(skewness_data, weighted = T) #Market cap weighted (rbind warning should be okay, just NAs)
univariate_sort_weighted_skewness

Warning message in rbind(deparse.level, ...):
"mismatched types: converting objects to numeric"


,1,2,3,4,5,5-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mean of Skewness,-1.11162615,-0.30518393,-0.04901578,0.23491231,1.20434155,NA
Mean of one-ahead returns,0.01782412,0.01876574,0.01628081,0.01742247,0.01635784,-0.001466283
T-statistic,4.34587103,3.71589047,3.12619832,3.48104449,3.69941250,-0.385749563
Alpha,0.02129359,0.02206418,0.01719598,0.02078340,0.01757056,-0.003723028
T-statistic (alpha),4.19598061,3.99173328,2.97015615,3.39924585,4.05182591,-0.881502776


### Bivariate sort

In [18]:
#Bivariate sort on size and beta
bivariate_sort_size_beta <- bivariate_sort(size, betas)
bivariate_sort_size_beta

Warning message in min(j, na.rm = TRUE):
"no non-missing arguments to min; returning Inf"
Warning message in max(j, na.rm = TRUE):
"no non-missing arguments to max; returning -Inf"


,Size 1,Size 2,Size 3,Size 3-1
,<dbl>,<dbl>,<dbl>,<dbl>
Beta 1 - Mean,0.021038210,0.014647184,0.015437239,-0.005600971
Beta 1 - T-statistic,5.132322750,4.418048185,4.602719306,-1.719324985
Beta 1 - Alpha,0.027218451,0.019793581,0.019863700,-0.007354751
Beta 1 - T-statistic (alpha),5.594380376,4.933462876,5.396942728,-2.581931780
Beta 2 - Mean,0.023056237,0.017810270,0.017859206,-0.005197031
Beta 2 - T-statistic,3.485829432,2.814828606,3.342261666,-1.961550620
Beta 2 - Alpha,0.024060296,0.018910377,0.018618823,-0.005441474
Beta 2 - T-statistic (alpha),3.496470085,2.936559765,3.571760018,-1.826449557
Beta 3 - Mean,0.034196337,0.021819556,0.018575170,-0.015621167


In [19]:
#Bivariate sort on size and skewness
bivariate_sort_size_skewness <- bivariate_sort(size, skewness_data)
bivariate_sort_size_skewness

,Size 1,Size 2,Size 3,Size 3-1
,<dbl>,<dbl>,<dbl>,<dbl>
Skewness 1 - Mean,0.023410815,1.815745e-02,0.0160117896,-0.007399025
Skewness 1 - T-statistic,4.166112707,4.086861e+00,3.7417714678,-2.525162042
Skewness 1 - Alpha,0.027666615,2.010950e-02,0.0193449122,-0.008321703
Skewness 1 - T-statistic (alpha),4.172345432,4.192339e+00,4.0406153292,-2.885248626
Skewness 2 - Mean,0.026899482,1.771002e-02,0.0176345678,-0.009264914
Skewness 2 - T-statistic,4.020958856,3.025633e+00,3.3441705863,-2.880581855
Skewness 2 - Alpha,0.028024849,2.183667e-02,0.0199746752,-0.008050174
Skewness 2 - T-statistic (alpha),4.019759490,2.993378e+00,3.5406438299,-2.642094304
Skewness 3 - Mean,0.031698930,1.814005e-02,0.0158233554,-0.015875574


### Fama-MacBeth regression

In [24]:
#Fama-Macbeth for skewness
FM_skewness <- fama_macbeth(list(Skewness = skewness_data))
FM_skewness

,(Intercept),Skewness,Adj. R-squared,n
,<dbl>,<dbl>,<dbl>,<dbl>
Mean,0.01576496,0.001733822,0.006028238,248.4459
T-statistic,3.14153586,1.047018454,NA,NA


In [25]:
#Fama-Macbeth for the three sort variables
indep_var <- list(Skewness = skewness_data, Size = size, Beta = betas)
FM_three_sorts <- fama_macbeth(indep_var)
FM_three_sorts

,(Intercept),Skewness,Size,Beta,Adj. R-squared,n
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mean,0.01724554,-0.0005697053,-0.003514242,0.01294675,0.09850542,244.0455
T-statistic,3.60827536,-0.7296489638,-4.158311507,1.79528130,NA,NA


## 5. Conclusion

## Appendix A

### Data preparation

We performed the following steps to obtain the final data utilized in our analyses. Firstly, we loaded the data on Adjusted Close prices and Market capitalization which we collected in Homework 1. In addition, we downloaded the Fama-French 3 factors as well as the Momentum factor from [here](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html). The last required piece of data was Liquidity which we accessed at the [website](https://faculty.chicagobooth.edu/lubos-pastor/data) of Ľuboš Pástor.

In [ ]:
#Loading the data
yahoo_data <- readRDS("Raw data/list_yahoo.RData") #Adjusted close prices and volumes
market_cap <- readRDS("Raw data/market_cap_data.RData") #Market capitalization
fffactors_monthly <- read_csv("Raw data/F-F_Research_Data_Factors.CSV", skip = 2, n_max = 1149) #Need to limit the number of rows since the file contains yearly data as well
fffactors_daily <- read_csv("Raw data/F-F_Research_Data_Factors_daily.CSV", skip = 3, n_max = 25210) #Last row is some copyright => skip
momentum <- read_csv("Raw data/F-F_Momentum_Factor.CSV", skip = 12, n_max  = 1144, na = c("", "NA", -99.99, -999))
liquidity <- read.delim("Raw data/liq_data_1962_2021.txt", skip = 10, na = c("", "NA", -99.99, -999, -99))

Next, we define an auxilliary function which removes empty lines from an xts object (for future use).

In [ ]:
#Defining a function to strip an xts object of lines with 0 observations (all NAs)
strip_empty_lines <- function(xts_object) {
  na_sums <- apply(xts_object, 1, function(x) {sum(is.na(x))})
  indic <- na_sums < ncol(xts_object)
  return(xts_object[indic])
}

Then, we extract the Adjusted Close price.

In [ ]:
#Extracting the Adjusted Close Price
daily_adj_close <- lapply(yahoo_data, "[", , 6) #The unused argument is not an error, we have to skip it to specify columns

In order to calculate the excess returns, we need the market risk premium. Therefore, we extract it from the Fama-French factors data set. We also convert the daily observations to monthly and save the resulting data for future use in the portfolio analysis.

In [ ]:
#Extracting market risk premium and the risk-free rate
fffactors_monthly <- as.data.frame(fffactors_monthly) #Converting from tibble to a data frame
fffactors_daily <- as.data.frame(fffactors_daily) #Converting from tibble to a data frame
colnames(fffactors_daily)[1] <- "Date"
colnames(fffactors_monthly)[1] <- "Date"
fffactors_daily$Date <- as.Date(as.character(fffactors_daily$Date), "%Y%m%d") #Creating a date (daily)
fffactors_monthly$Date <-  as.Date(paste0(as.character(fffactors_monthly$Date),"01"), "%Y%m%d") + months(1) - days(1) #Creating a date (end of month)
fffactors_monthly[, 2:5] <- fffactors_monthly[, 2:5] / 100 #Scale by 100 since the data is in percents
fffactors_daily[, 2:5] <- fffactors_daily[, 2:5] / 100 #Scale by 100 since the data is in percents
market_prem_daily <- xts(fffactors_daily$`Mkt-RF`, order.by = fffactors_daily$Date)
names(market_prem_daily) <- "Mkt_RF_daily"
market_prem_monthly <- xts(fffactors_monthly$`Mkt-RF`, order.by = fffactors_monthly$Date)
names(market_prem_monthly) <- "Mkt_RF_monthly"
rf_daily <- xts(fffactors_daily$RF, order.by = fffactors_daily$Date)
names(rf_daily) <- "RF_daily"
rf_monthly <- xts(fffactors_monthly$RF, order.by = fffactors_monthly$Date)
names(rf_monthly) <- "RF_monthly"
fffactors_monthly_xts <- as.xts(fffactors_monthly[, 2:5], order.by = fffactors_monthly$Date) #Converting monthly factors to xts
fffactors_monthly_xts <- fffactors_monthly_xts["2007/"] #Restricting the period since we do not need older data
#saveRDS(fffactors_monthly_xts, file = "Final data/fffactors_monthly.RData") #Saving for future use

Now we can calculate the daily and monthly excess returns. The former will be used for the calculation of various measures such as beta and skewness. We save the latter for the portfolio analysis

In [ ]:
#Calculating excess returns
calc_excess_returns <- function(xts_object, daily = T) { #Defining a function to calculate excess returns
  if (daily == T) { #For daily returns
    result <- dailyReturn(xts_object)
    result <- result[2:nrow(result)] #Disregard the first observation (dailyReturn sets it to 0)
    risk_free_rate <- rf_daily[index(result)] #Define daily risk free rate
  } else { #For monthly returns
    result <- monthlyReturn(xts_object)
    risk_free_rate <- rf_monthly[index(result)]
  }
  return(result - risk_free_rate) #Substract risk free rate from the returns
}
#Daily excess returns
daily_returns <- lapply(daily_adj_close, calc_excess_returns) #Applying the function
daily_return_names <- gsub(".Adjusted", ".DailyReturns", unlist(lapply(daily_adj_close, names))) #New names
for (i in 1:length(daily_returns)) { #Renaming the columns for clarity
  names(daily_returns[[i]]) <- daily_return_names[i]
}
#Monthly excess returns
monthly_returns <- lapply(daily_adj_close, calc_excess_returns, daily = F)
monthly_return_names <- gsub(".Adjusted", ".MonthlyReturns", unlist(lapply(daily_adj_close, names))) #New names
for (i in 1:length(monthly_returns)) { #Renaming the columns for clarity
  names(monthly_returns[[i]]) <- monthly_return_names[i]
}
#Merge monthly excess returns
monthly_returns_merged <- do.call(merge.xts, monthly_returns)
#saveRDS(monthly_returns_merged, file = "Final data/monthly_excess_returns.RData") #Saving for future use

In the following step we define a function that calculates a specified monthly measure from past 12 months of daily data. In order to calculate the measure in a given period, we require at least 200 data points which is a standard for 12 months of daily data.

In [ ]:
#Defining a generic function to calculate a monthly measure from past 12 months of daily data
calc_measure <- function(xts_object, func, measure_name) { #Expects a series of monthly returns, the function to calculate the required measure (skewness, beta..) and the name of the measure
  result <- xts(rep(NA, nrow(xts_object)), order.by = index(xts_object)) #Empty xts object for the results
  names(result) <- gsub("MonthlyReturns", measure_name, names(xts_object)) #Naming the resulting object for clarity 
  ind <- which(monthly_return_names == names(xts_object)) #Get the rank of the stock in the list (1-250)
  for (i in 1:nrow(xts_object)) { #Loop through the rows of the object
    end_date <- index(xts_object)[i] #The last day that the daily returns will be used for computations
    start_date <- end_date %m-% months(12) #The start date is 12 months back as required
    df_to_use <- daily_returns[[ind]][paste0(start_date, "/", end_date)] #Extracting the subset we will use for the computation (12 months of daily returns)
    if (length(df_to_use) < 200){ #We require at least 200 data points, otherwise we won't calculate
      next
    } else {
      result[i] <- func(na.omit(df_to_use)) #Apply the specified function
    }
  }
  return(result)
}

We then apply the function on monthly returns for each of the 250 stocks. The first measure we calculate is Skewness (It is the third sorting variable that was assigned to our group). We save the result for future use.

In [ ]:
#Calculating skewness
skewness_data <- lapply(monthly_returns, calc_measure, func = skewness, measure_name = "Skewness") #Applying the function (skewness from the moments package)
#Merging skewness data
skewness_data_merged <- do.call(merge.xts, skewness_data)
skewness_data_merged <- strip_empty_lines(skewness_data_merged) #Remove empty lines from data (periods with no data)
#saveRDS(skewness_data_merged, file = "Final data/skewness.RData") #Saving for future use

Another sorting variable that we utilize in the portfolio analysis is Beta. We define a function that estimates beta for a given time series and then we pass it as an argument to the function above.

In [ ]:
#Calculating betas
calc_beta <- function(xts_object) { #Defining a function to calculate the beta (based on a series of daily returns)
  model_res <- lm(xts_object ~ market_prem_daily[index(xts_object)]) #Regress market premium on daily excess returns
  return(model_res$coefficients[[2]]) #Extract the estimated coefficient on market premium (beta)
}
#Applying the function
betas <- lapply(monthly_returns, calc_measure, func = calc_beta, measure_name = "Beta")
#Merging the betas
betas_merged <- do.call(merge.xts, betas)
betas_merged <- strip_empty_lines(betas_merged)
#saveRDS(betas_merged, file = "Final data/betas.RData") #Saving for future use

The last sorting variable we will use is Size. It can be calculated directly from Market Capitalization. We save the data for both of those factors.

In [ ]:
#Extracting monthly market cap data
market_cap_monthly <- lapply(market_cap, to.monthly, OHLC = F, indexAt = "lastof") #Converting daily to monthly
market_cap_monthly_merged <- do.call(merge.xts, market_cap_monthly) #Merging
market_cap_monthly_merged <- strip_empty_lines(market_cap_monthly_merged)
#saveRDS(market_cap_monthly_merged, file = "Final data/market_cap_monthly.RData") #Saving for future use

#Calculating size
size <- log(market_cap_monthly_merged)
names(size) <- gsub(".Adjusted", ".Size", unlist(lapply(daily_adj_close, names))) #New names
size <- as.xts(apply(size, 2, function(x) {ifelse(is.finite(x), x, NA)}), order.by = index(size)) #Replace -Inf with NA
size <- strip_empty_lines(size) #Remove empty rows
#saveRDS(size, file = "Final data/size.RData") #Saving for future use

We also extract the factors Momentum and Liquidity which are essential for the calculation of $\alpha$.

In [ ]:
#Extracting momentum
momentum <- as.data.frame(momentum)
colnames(momentum)[1] <- "Date"
momentum$Date <-  as.Date(paste0(as.character(momentum$Date[1]),"01"), "%Y%m%d") + months(1) - days(1) #Creating a date (end of month)
momentum_xts <- xts(momentum[, 2], order.by = momentum$Date) #Converting to xts
names(momentum_xts) <- "Momentum" #Renaming
momentum_xts <- momentum_xts["2007/"] #Restricting the period
#saveRDS(momentum_xts, "Final data/momentum.RData") #Saving for future use

#Extracting liquidity
colnames(liquidity)[1] <- "Date"
liquidity$Date <-  as.Date(paste0(as.character(liquidity$Date),"01"), "%Y%m%d") + months(1) - days(1) #Creating a date (end of month)
liquidity_xts <- xts(liquidity[, 4], order.by = liquidity$Date) #Converting to xts
names(liquidity_xts) <- "Liquidity" #Renaming
liquidity_xts <- liquidity_xts["2007/"] #Restricting the period
#saveRDS(liquidity_xts, "Final data/liquidity.RData") #Saving for future use

## Appendix B